In [74]:
import sys
# import mkl
# mkl.set_num_threads(30)
import shutil

import os
import imp
os.environ['CUDA_VISIBLE_DEVICES'] ="2"

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
#import tensorflow_hub as hub

import math

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns

from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

import utils
import config
import image
import dataset
import model
import IPython
#import extract_flow
#import extract_RGBframes

from utils import getImagePath

from collections import defaultdict
from tqdm import tqdm

import numpy as np
import cv2
from glob import glob
from multiprocessing import Pool
from sklearn.preprocessing import MinMaxScaler

imp.reload(config)
imp.reload(image)
imp.reload(utils)
imp.reload(dataset)
imp.reload(model)
#imp.reload(extract_flow)
#imp.reload(extract_RGBframes)


if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [2]:
tf.config.list_physical_devices('GPU')

2022-10-16 02:14:08.372887: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-10-16 02:14:08.373023: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: midway2-0606.rcc.local
2022-10-16 02:14:08.373069: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: midway2-0606.rcc.local
2022-10-16 02:14:08.373351: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 460.32.3
2022-10-16 02:14:08.373457: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 460.32.3
2022-10-16 02:14:08.373471: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 460.32.3


[]

In [3]:
images_metadata_df = dataset.load_images_metadata()
images_metadata_birds_df = images_metadata_df.loc[images_metadata_df['obj_cat_binary']==1].sample(frac = 1, random_state = 42)

images_metadata_dict = dataset.load_images_metadata_dict(images_metadata_birds_df)

In [4]:
twostream_dict = images_metadata_dict.copy()

In [5]:
isFile = os.path.exists('/project2/msca/xtang36/cap_test_rgb/29285')
print(isFile)

True


In [6]:
for obj_id, obj_info in twostream_dict.items():
    data_path_v = os.path.join(*[str(x) for x in [config.DATA_DIR, obj_info['day_dir'], obj_info['camera_dir'], 
                             obj_info['video_dir'], obj_info['track_dir']]])
    frame_path_list = [os.path.join(data_path_v, file_path) for file_path in obj_info['image_file']]
    #del optical_dict[obj_id]['video_path']
    twostream_dict[obj_id]['image_path'] = data_path_v
    #optical_dict['frame_']
#twostream_dict[29285]

### Extract New frames

In [14]:
### Extract Optical
def compute_TVL1(prev, curr, bound=15):
    """Compute the TV-L1 optical flow."""
    TVL1 = cv2.optflow.DualTVL1OpticalFlow_create()
    flow = TVL1.calc(prev, curr, None)
    #flow_frame = np.clip(flow_frame, -20, 20)
    assert flow.dtype == np.float32

    flow = (flow + bound) * (255.0 / (2*bound))
    flow = np.round(flow).astype(int)
    flow[flow >= 255] = 255
    flow[flow <= 0] = 0

    return flow

def cal_for_frames(video_path):
    #print(video_path)
    frames = glob(os.path.join(video_path, '*.png'))
    frames.sort()
    #print(frames)
    flow = []
    prev = cv2.imread(frames[0])
    prev = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
    for i, frame_curr in enumerate(frames):
        curr = cv2.imread(frame_curr)
        curr = cv2.cvtColor(curr, cv2.COLOR_BGR2GRAY)
        tmp_flow = compute_TVL1(prev, curr)
        flow.append(tmp_flow)
        prev = curr

    return flow

def save_flow(video_flows, flow_path):
    for i, flow in enumerate(video_flows):
        cv2.imwrite(os.path.join(flow_path.format('u'), "{}_{:06d}.jpg".format(obj_id, i)), flow[:, :, 0])
        cv2.imwrite(os.path.join(flow_path.format('v'), "{}_{:06d}.jpg".format(obj_id, i)), flow[:, :, 1])
        
def extract_flow(args):
    video_path, flow_path = args
    flow = cal_for_frames(video_path)
    save_flow(flow, flow_path)
    #print('complete:' + flow_path)
    return

In [15]:
### Extract RGB
def cal_for_RGB_frames(video_path):
    #print(video_path)
    frames = glob(os.path.join(video_path, '*.png'))
    frames.sort()
    #print(frames)
    flow = []
    for f in frames:
        rgb_frame = cv2.imread(f)
        rgb_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_BGR2HSV)
        flow.append(rgb_frame)

    return flow

def save_flow(video_flows, flow_path):
    for i, flow in enumerate(video_flows):
        cv2.imwrite(os.path.join(flow_path.format('u'), "{}_{:06d}.jpg".format(obj_id, i)), flow[:, :, 0])
        cv2.imwrite(os.path.join(flow_path.format('v'), "{}_{:06d}.jpg".format(obj_id, i)), flow[:, :, 1])
        
def extract_RGBframes(args):
    video_path, flow_path = args
    flow = cal_for_RGB_frames(video_path)
    save_flow(flow, flow_path)
    #print('complete:' + flow_path)
    return

In [9]:
twostream_dict[29285]

{'day_dir': '2020-09-02-mw2',
 'camera_dir': 'dnncame',
 'video_dir': 'video-00112-2020_08_31_21_58_14',
 'track_dir': 321,
 'directory_x': '/video-00112-2020_08_31_21_58_14/321',
 'count': 1,
 'id': 321,
 'directory_y': '/video-00112-2020_08_31_21_58_14/321',
 'bird': 1,
 'cable': 0,
 'panel': 0,
 'plant': 0,
 'car': 0,
 'human': 0,
 'other_animal': 0,
 'insect': 0,
 'aircraft': 0,
 'other': 0,
 'unknown': 0,
 'fly_over_above': 1,
 'fly_over_reflection': 0,
 'fly_through': 0,
 'perch_on_panel': 0,
 'land_on_ground': 0,
 'perch_in_background': 0,
 'collision': 0,
 'uncertain': 0,
 'image_count': 50,
 'obj_cat': 1,
 'obj_cat_binary': 1,
 'activity_cat': 1,
 'ttv_split': 1.0,
 'image_file': ['video-00112-2020_08_31_21_58_14_8771_916_632_42.32322_13456_321_275.1944289077348_272.116035335796_[44.18144407].png',
  'video-00112-2020_08_31_21_58_14_8751_1572_644_22.54145_12769_321_272.3859440303888_250.642313350809_[24.0208243].png',
  'video-00112-2020_08_31_21_58_14_8766_1124_624_36.82334_1

### New Frame Generation -- SKIP FOR NOW

In [16]:
rgb_flow_path='/project2/msca/xtang36/cap_test_rgb/'
optical_flow_path='/project2/msca/xtang36/cap_test_optical/'
n_rgb = 0
n_opt = 0
#add rgb image path to the data_dictionary
for obj_id, obj_info in twostream_dict.items():
    if n_rgb <=8000:

        directory = str(obj_id)
        new_path = os.path.join(rgb_flow_path, directory)
        isFile = os.path.exists(new_path)
        if isFile == True:
            #print("path", new_path, "is already exists")
            pass
        else:
            #print(new_path)
            os.mkdir(new_path)
        opt_input_path = obj_info['image_path']
        args = opt_input_path, new_path
        extract_RGBframes(args)
        opt_output_frames = glob(os.path.join(new_path, '*.jpg'))
        opt_output_frames.sort()
        twostream_dict[obj_id]['rgb_files'] = opt_output_frames
        n_rgb+=1
    else:
        break

print('RBG done')
#add optical flow image path to the data_dictionary
for obj_id, obj_info in twostream_dict.items():
    if n_opt <=8000:
        directory = str(obj_id)
        new_path = os.path.join(optical_flow_path, directory)
        isFile = os.path.exists(new_path)
        if isFile == True:
            #print("path", new_path, "is already exists")
            pass
        else:
            #print(new_path)
            os.mkdir(new_path)
        opt_input_path = obj_info['image_path']
        args = opt_input_path, new_path
        extract_flow(args)
        opt_output_frames = glob(os.path.join(new_path, '*.jpg'))
        opt_output_frames.sort()
        twostream_dict[obj_id]['optical_files'] = opt_output_frames
        n_opt +=1
    else:
        break

KeyboardInterrupt: 

In [27]:
#Get rgb and optical flow images from file path into the data dict
for obj_id, obj_info in twostream_dict.items():
    data_path_opt = os.path.join("/project2/msca/xtang36/cap_test_optical", str(obj_id))
    data_path_rgb = os.path.join("/project2/msca/xtang36/cap_test_rgb", str(obj_id))
    isFile_opt = os.path.exists(data_path_opt)
    isFile_rgb = os.path.exists(data_path_rgb)
    if isFile_opt == True:
        opt_output_frames = glob(os.path.join(data_path_opt, '*.jpg'))
        opt_output_frames.sort()
        twostream_dict[obj_id]['optical_files'] = opt_output_frames
        #print('optical flow frames are added to the dict')
    else:
        pass

    if isFile_rgb == True:
        rgb_output_frames = glob(os.path.join(data_path_rgb, '*.jpg'))
        rgb_output_frames.sort()
        twostream_dict[obj_id]['rgb_files'] = rgb_output_frames
        #print('rgb frames are added to the dict')
    else:
        pass

In [21]:
len(twostream_dict)

17155

In [18]:
twostream_dict[29285]

{'day_dir': '2020-09-02-mw2',
 'camera_dir': 'dnncame',
 'video_dir': 'video-00112-2020_08_31_21_58_14',
 'track_dir': 321,
 'directory_x': '/video-00112-2020_08_31_21_58_14/321',
 'count': 1,
 'id': 321,
 'directory_y': '/video-00112-2020_08_31_21_58_14/321',
 'bird': 1,
 'cable': 0,
 'panel': 0,
 'plant': 0,
 'car': 0,
 'human': 0,
 'other_animal': 0,
 'insect': 0,
 'aircraft': 0,
 'other': 0,
 'unknown': 0,
 'fly_over_above': 1,
 'fly_over_reflection': 0,
 'fly_through': 0,
 'perch_on_panel': 0,
 'land_on_ground': 0,
 'perch_in_background': 0,
 'collision': 0,
 'uncertain': 0,
 'image_count': 50,
 'obj_cat': 1,
 'obj_cat_binary': 1,
 'activity_cat': 1,
 'ttv_split': 1.0,
 'image_file': ['video-00112-2020_08_31_21_58_14_8771_916_632_42.32322_13456_321_275.1944289077348_272.116035335796_[44.18144407].png',
  'video-00112-2020_08_31_21_58_14_8751_1572_644_22.54145_12769_321_272.3859440303888_250.642313350809_[24.0208243].png',
  'video-00112-2020_08_31_21_58_14_8766_1124_624_36.82334_1

In [41]:
data_dir = "/project2/msca/xtang36/cap_test_rgb"
obj_id_ = 29285

frame_path_list = [os.path.join(*[data_dir, str(obj_id) ,file_path]) for file_path in twostream_dict[obj_id_]['rgb_files']]
print(len(frame_path_list))

58


In [26]:
def generator_dataset_activity_ts_rgb(obj_metadata_dict):
    data_dir = "/project2/msca/xtang36/cap_test_rgb"
    for obj_id, obj_info in obj_metadata_dict.items():
        
        data_path_rgb = os.path.join(data_dir, str(obj_id))
        isFile_rgb = os.path.exists(data_path_rgb)
        if isFile_opt == True:
            frame_path_list = [os.path.join(*[data_dir, str(obj_id) ,file_path]) for file_path in obj_info['rgb_files']]
            # Counter
            num_total_frames = len(frame_path_list) # the number of imgs in this track
            num_used_frames = min(config.NUM_INPUT_FRAME, num_total_frames)
            num_blank_frames = config.NUM_INPUT_FRAME - num_used_frames
            num_skip_at_begining = max(0, (num_total_frames-config.NUM_INPUT_FRAME) >> 1)

            # Frames
            frames = np.zeros([config.NUM_INPUT_FRAME, config.IMG_HEIGHT,  config.IMG_WIDTH,  config.NUM_CHANNEL])
            frames[:num_used_frames] = tf.image.resize(
                np.stack([
                    image.load(frame_path_list[num_skip_at_begining+frame_idx]) 
                    for frame_idx in range(num_used_frames)]
                ), [config.IMG_HEIGHT, config.IMG_WIDTH])

            frames = tf.convert_to_tensor(frames, tf.float32)
            # Labels
            label = obj_info['activity_cat']
            yield frames, label
        else:
            pass

In [34]:
def generator_dataset_activity_ts_opt(obj_metadata_dict):
    data_dir = "/project2/msca/xtang36/cap_test_optical"
    for obj_id, obj_info in obj_metadata_dict.items():
        
        data_path_opt = os.path.join(data_dir, str(obj_id))
        isFile_opt = os.path.exists(data_path_opt)
        if isFile_opt == True:
            frame_path_list = [os.path.join(*[data_dir, str(obj_id) ,file_path]) for file_path in obj_info['optical_files']]
            # Counter
            num_total_frames = len(frame_path_list) # the number of imgs in this track
            num_used_frames = min(config.NUM_INPUT_FRAME, num_total_frames)
            num_blank_frames = config.NUM_INPUT_FRAME - num_used_frames
            num_skip_at_begining = max(0, (num_total_frames-config.NUM_INPUT_FRAME) >> 1)

            # Frames
            frames = np.zeros([config.NUM_INPUT_FRAME, config.IMG_HEIGHT,  config.IMG_WIDTH,  config.NUM_CHANNEL])
            frames[:num_used_frames] = tf.image.resize(
                np.stack([
                    image.load(frame_path_list[num_skip_at_begining+frame_idx]) 
                    for frame_idx in range(num_used_frames)]
                ), [config.IMG_HEIGHT, config.IMG_WIDTH])

            frames = tf.convert_to_tensor(frames, tf.float32)
            # Labels
            label = obj_info['activity_cat']
            yield frames, label
        else:
            pass

In [43]:
dict_dataset_rgb = {'train':dict(), 'valid':dict(), 'test':dict()}
for obj_id, obj_info in twostream_dict.items():
    if 'rgb_files' in obj_info.keys():
        ttv_split = obj_info['ttv_split']
        if ttv_split == 1: dict_dataset_rgb['train'][obj_id] = obj_info
        elif ttv_split == 2: dict_dataset_rgb['valid'][obj_id] = obj_info
        elif ttv_split == 3: dict_dataset_rgb['test'][obj_id] = obj_info
        else: raise ValueError()
    else:
        pass

dict_dataset_opt = {'train':dict(), 'valid':dict(), 'test':dict()}
for obj_id, obj_info in twostream_dict.items():
    if 'optical_files' in obj_info.keys():
        ttv_split = obj_info['ttv_split']
        if ttv_split == 1: dict_dataset_opt['train'][obj_id] = obj_info
        elif ttv_split == 2: dict_dataset_opt['valid'][obj_id] = obj_info
        elif ttv_split == 3: dict_dataset_opt['test'][obj_id] = obj_info
        else: raise ValueError()
    else:
        pass

In [46]:
{key:len(value) for key,value in dict_dataset_opt.items()}

{'train': 5938, 'valid': 1236, 'test': 1275}

In [47]:
{key:len(value) for key,value in dict_dataset_rgb.items()}

{'train': 5629, 'valid': 1175, 'test': 1197}

In [48]:
ts_dataset_rgb = {key:tf.data.Dataset.from_generator(lambda: generator_dataset_activity_ts_rgb(value), 
                                       output_types= config.DATASET_TYPE,
                                       output_shapes = config.DATASET_SHAPE) 
              for key,value in dict_dataset_rgb.items()}
ts_dataset_opt = {key:tf.data.Dataset.from_generator(lambda: generator_dataset_activity_ts_opt(value), 
                                       output_types= config.DATASET_TYPE,
                                       output_shapes = config.DATASET_SHAPE) 
              for key,value in dict_dataset_opt.items()}
ts_dataset_rgb = {key:dataset.configure_for_performance(value) for key,value in ts_dataset_rgb.items()}
ts_dataset_opt = {key:dataset.configure_for_performance(value) for key,value in ts_dataset_opt.items()}

In [49]:
ts_dataset_rgb

{'train': <BatchDataset element_spec=(TensorSpec(shape=(None, None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>,
 'valid': <BatchDataset element_spec=(TensorSpec(shape=(None, None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>,
 'test': <BatchDataset element_spec=(TensorSpec(shape=(None, None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>}

### Define Models

In [70]:
from keras.layers.convolutional import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD, Adam
from keras.layers.normalization.batch_normalization import BatchNormalization
from keras.applications.inception_v3 import InceptionV3
from keras.models import Sequential, load_model, Model
from keras.layers import Input, Average, GlobalAveragePooling2D
from keras.layers.core import Dense, Dropout, Activation, Flatten

#### RGB CNN  - Spatial

In [71]:

def RGB_Model(data, weights = 'imagenet'):
    
    base_model = InceptionV3(weights=weights, include_top=False)

    x = base_model.output
    x = GlobalAveragePooling2D()(x)

    x = Dense(1024, activation='relu')(x)

    predictions = Dense(len(config.ACT_LIST), activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    return model

#### Optical Flow CNN - Temporal

In [72]:
def cnn_temporal():
    model = Sequential()
    model.add(Conv2D(96, (7, 7), strides=2, padding='same', input_shape=config.IMG_SIZE))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, (5, 5), strides=2, padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(512, (3, 3), strides=1, activation='relu', padding='same'))

    model.add(Conv2D(512, (3, 3), strides=1, activation='relu', padding='same'))

    model.add(Conv2D(512, (3, 3), strides=1, activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.9))

    model.add(Dense(2048, activation='relu'))
    model.add(Dropout(0.9))

    model.add(Dense(self.nb_classes, activation='softmax'))

    return model

### Fit Models

#### RGB

In [75]:
rgb_model = RGB_Model(ts_dataset_rgb['train'])

rgb_model.compile(
    optimizer='rmsprop', 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)


rgb_model.fit(
    ts_dataset_rgb['train'],
    epochs=10,
    batch_size=config.BATCH_SIZE,
    steps_per_epoch = 6000 // config.BATCH_SIZE + 1,
    validation_data=ts_dataset_rgb['valid'],
    callbacks = [
                 keras.callbacks.EarlyStopping(patience = 2, restore_best_weights = True, monitor = 'val_accuracy'),
                 keras.callbacks.ModelCheckpoint("ts_rgb_ft.h5", save_best_only=True), 
                 keras.callbacks.TensorBoard(utils.get_run_logdir(log_name = 'ts_rgb_ft'))
             ]
)

NameError: name 'shutil' is not defined